<a href="https://colab.research.google.com/github/Yago-Coqueiro/ReActAgente-FastCamp/blob/main/ReAct_Agente_Aula.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install groq #instalação do groq


In [18]:
import os
os.environ['GROQ_API_KEY'] = "Coloque sua chave API aqui" #Configuração do chave API como variável de ambiente

AGENTE SEM LOOP AUTOMÁTICO:

In [3]:
# TESTE DE MODELO E API

import os
from groq import Groq

# Inicialização do cliente e busca automática da chave 'GROQ_API_KEY' definida no ambiente
client = Groq()

# Chamada para geração de texto
completion = client.chat.completions.create(
    model="llama-3.3-70b-versatile",     # Seleção do modelo

    # Define a estrutura da conversa (histórico)
    messages=[
        {
            # 'role: user' identifica que esta é a entrada do usuário final
            "role": "user",
            "content": "Explain why fast inference is critical for reasoning models"
        }
    ]
)

# Acessa o objeto de resposta, entra na lista 'choices' e extrai o texto (.content)
print(completion.choices[0].message.content)

Fast inference is critical for reasoning models for several reasons:

1. **Real-time Decision Making**: In many applications, such as autonomous vehicles, robotics, or real-time expert systems, reasoning models need to make decisions quickly to respond to changing situations. Fast inference enables these models to keep up with the pace of the environment and make timely decisions.
2. **Scalability**: As the size and complexity of reasoning models increase, the time it takes to perform inference can become a bottleneck. Fast inference allows these models to scale up to larger problem sizes and more complex reasoning tasks without becoming impractically slow.
3. **User Experience**: In interactive applications, such as chatbots, virtual assistants, or decision support systems, users expect rapid responses to their queries. Fast inference enables these systems to provide answers quickly, improving the overall user experience and engagement.
4. **Resource Efficiency**: Fast inference can h

In [4]:
# Classe que representa o agente
class Agent:
  def __init__(self, client, system):
    self.client = client     # Cliente da API
    self.system = system     # System Prompt
    self.messages = []       # Histórico de mensagens

    # Define o System Prompt como primeira mensagem (guia)
    if self.system is not None:
      self.messages.append({"role": "system", "content": self.system})

  # Função de memória do agente
  def __call__(self, message=""):
    if message:
      self.messages.append({"role": "user", "content": message})    # Adição da mensagem do usuário ao histórico
      result = self.execute()                                       # Execução para obter resposta
      self.messages.append({"role": "assistant", "content": result})  # Adição da mensagem da IA ao histórico
      return result

  def execute(self):
    # Finalização de chat solicitada
    completion = client.chat.completions.create(
        model  = "llama-3.3-70b-versatile",  # Seleção de modelo
        messages = self.messages               # Lista completa de mensagem acumulada na instância
    )
    return completion.choices[0].message.content


In [5]:
# SYSTEM PROMPT:(Guia para o modelo)
system_prompt = """
You run in a loop of Thought, Action, PAUSE, Observation.
At the end of the loop you output an Answer
Use Thought to describe your thoughts about the question you have been asked.
Use Action to run one of the actions available to you - then return PAUSE.
Observation will be the result of running those actions.

Your available actions are:

calculate:
e.g. calculate: 4 * 7 / 3
Runs a calculation and returns the number - uses Python so be sure to use floating point syntax if necessary

get_planet_mass:
e.g. get_planet_mass: Earth
returns weight of the planet in kg

Example session:

Question: What is the mass of Earth times 2?
Thought: I need to find the mass of Earth
Action: get_planet_mass: Earth
PAUSE

You will be called again with this:

Observation: 5.972e24

Thought: I need to multiply this by 2
Action: calculate: 5.972e24 * 2
PAUSE

You will be called again with this:

Observation: 1,1944×10e25

If you have the answer, output it as the Answer.

Answer: The mass of Earth times 2 is 1,1944×10e25.

Now it's your turn:
""".strip()

# Ferramenta de cálculo
def calculate(operation: str) -> float:
    return eval(operation)

# Ferramenta de pesquisa de massa de planetas
def get_planet_mass(planet) -> float:
    planet = planet.lower().strip()
    if planet == "earth":
        return 5.972e24
    if planet == "mars":
        return 6.39e23
    if planet == "jupiter":
        return 1.898e27
    if planet == "saturn":
        return 5.683e26
    if planet == "uranus":
        return 8.681e25
    if planet == "neptune":
        return 1.024e26
    if planet == "mercury":
        return 3.285e23
    if planet == "venus":
        return 4.867e24
    return None

In [6]:
# Inicializando o agente
neil_tyson = Agent(client, system_prompt)

In [7]:
# Solicitação inicial (Pensamento + Ação)
result = neil_tyson("What is the mass of Mercury times 5?")
print(result)

Thought: To find the mass of Mercury times 5, I first need to determine the mass of Mercury. 

Action: get_planet_mass: Mercury
PAUSE


In [8]:
# Continuidade Loop (Uso da ferramenta de pesquisa)
result = get_planet_mass("mercury")
print(result)

3.285e+23


In [9]:
# Continuidade Loop (Atribuição de observação para a resposta)
next_prompt = f"Observation: {result}"
next_prompt

'Observation: 3.285e+23'

In [10]:
# Continuidade Loop (segundo ciclo de Pensamento + Ação)
result = neil_tyson(next_prompt)
print(result)

Thought: Now that I have the mass of Mercury, I can calculate the mass of Mercury times 5 by multiplying the observed mass by 5.

Action: calculate: 3.285e+23 * 5
PAUSE


In [11]:
# Continuidade Loop (Uso da ferramenta de cálculo)
result = calculate("3.285e23 * 5")
print(result)

1.6425e+24


In [12]:
# Continuidade Loop (Segunda Atribuição de observação para a resposta)
next_prompt = f"Observation: {result}"
next_prompt

'Observation: 1.6425e+24'

In [13]:
# Encerramento do loop (Pensamento final e resposta ao usuário)
result = neil_tyson(next_prompt)
result

'Thought: I have now calculated the mass of Mercury times 5, so I can output the result as the final answer.\n\nAnswer: The mass of Mercury times 5 is 1.6425e+24.'

In [14]:
for msg in neil_tyson.messages:
    print(msg['content'])

You run in a loop of Thought, Action, PAUSE, Observation.
At the end of the loop you output an Answer
Use Thought to describe your thoughts about the question you have been asked.
Use Action to run one of the actions available to you - then return PAUSE.
Observation will be the result of running those actions.

Your available actions are:

calculate:
e.g. calculate: 4 * 7 / 3
Runs a calculation and returns the number - uses Python so be sure to use floating point syntax if necessary

get_planet_mass:
e.g. get_planet_mass: Earth
returns weight of the planet in kg

Example session:

Question: What is the mass of Earth times 2?
Thought: I need to find the mass of Earth
Action: get_planet_mass: Earth
PAUSE 

You will be called again with this:

Observation: 5.972e24

Thought: I need to multiply this by 2
Action: calculate: 5.972e24 * 2
PAUSE

You will be called again with this: 

Observation: 1,1944×10e25

If you have the answer, output it as the Answer.

Answer: The mass of Earth times 2 

In [15]:
# Histórico de mensagens
neil_tyson.messages


[{'role': 'system',
  'content': "You run in a loop of Thought, Action, PAUSE, Observation.\nAt the end of the loop you output an Answer\nUse Thought to describe your thoughts about the question you have been asked.\nUse Action to run one of the actions available to you - then return PAUSE.\nObservation will be the result of running those actions.\n\nYour available actions are:\n\ncalculate:\ne.g. calculate: 4 * 7 / 3\nRuns a calculation and returns the number - uses Python so be sure to use floating point syntax if necessary\n\nget_planet_mass:\ne.g. get_planet_mass: Earth\nreturns weight of the planet in kg\n\nExample session:\n\nQuestion: What is the mass of Earth times 2?\nThought: I need to find the mass of Earth\nAction: get_planet_mass: Earth\nPAUSE \n\nYou will be called again with this:\n\nObservation: 5.972e24\n\nThought: I need to multiply this by 2\nAction: calculate: 5.972e24 * 2\nPAUSE\n\nYou will be called again with this: \n\nObservation: 1,1944×10e25\n\nIf you have the

AGENTE COM LOOP AUTOMÁTICO:

In [16]:
import re

# Inicializa novo agente com loop automático
def agent_loop(max_iterations, system, query):
  agent = Agent(client, system_prompt)       # Agente aproveita as regras já estabelecidas
  tools = ['calculate', 'get_planet_mass']   # Define as ferramentas disponivéis
  next_prompt = query                        # Prompt da primeira iteração
  i = 0                                      # Contador de iterações

  # Laço de repetição para conduzir o ciclo do agente
  while i < max_iterations:
    i += 1
    result = agent(next_prompt)
    print(result)

    # Verifica a solicitações do uso de ferramentas e extrai seu nome
    if "PAUSE" in result and "Action" in result:
      action = re.findall(r"Action: ([a-z_]+): (.+)", result, re.IGNORECASE)
      chosen_tool = action[0][0]      # Seleção da ferramenta
      arg = action[0][1]              # Seleção do itea para aplicar a ferramenta

      # Verifica a existência da ferramenta e a executa resultando na observação
      if chosen_tool in tools:
        result_tool = eval(f"{chosen_tool}('{arg}')")
        next_prompt = f"Observation: {result_tool}"

      # Resposta para ferramenta não listada
      else:
        netx_prompt = "Observation: tool not found."

      # Garante visualização do processo
      print(next_prompt)
      continue

    # Em caso de resposta o loop se encerra
    if "Answer" in result:
      break

In [17]:
# Execução do agente com loop automático
agent_loop(max_iterations = 10, system = system_prompt, query="What is the mass of Earth plus the mass of Saturn and all of that times 2?")

Thought: To solve this problem, I need to find the mass of Earth and the mass of Saturn, add them together, and then multiply the result by 2. First, I'll find the mass of Earth.

Action: get_planet_mass: Earth
PAUSE
Observation: 5.972e+24
Thought: Now that I have the mass of Earth, I need to find the mass of Saturn.

Action: get_planet_mass: Saturn
PAUSE
Observation: 5.683e+26
Thought: Now that I have the masses of both Earth and Saturn, I can add them together and then multiply the result by 2. First, I'll add the masses of Earth and Saturn.

Action: calculate: 5.972e24 + 5.683e26
PAUSE
Observation: 5.74272e+26
Thought: Now that I have the sum of the masses of Earth and Saturn, I can multiply this result by 2 to get the final answer.

Action: calculate: 5.74272e+26 * 2
PAUSE
Observation: 1.148544e+27
Thought: I have now calculated the mass of Earth plus the mass of Saturn and multiplied the result by 2, so I have all the information I need to provide the final answer.

Answer: The ma